In [77]:
#import libraries 
from sklearn.model_selection import train_test_split 
from sklearn import svm
from sklearn import metrics

In [78]:
#extract dataset as pandas dataframe 
import pandas as pd
import numpy as np

cols_list = ['temperature', 'toux', 'rythme respiratoire']
symptoms = pd.read_csv ('dataset.csv', usecols=cols_list)
results = pd.read_csv ('dataset.csv', usecols=['target'])
print(symptoms) 

       temperature  toux  rythme respiratoire
0        38.909697     0            25.622143
1        38.501598     0            27.080123
2        36.262807     1            21.746695
3        36.432818     0            19.021566
4        39.146504     1            23.887650
...            ...   ...                  ...
24995    38.113612     1            28.226925
24996    36.948299     1            18.326012
24997    36.640686     0            19.775941
24998    38.723000     1            24.836507
24999    37.556375     1            28.816488

[25000 rows x 3 columns]


In [79]:
# splitting dataset , 80% for training and 20% for test
X_train, X_test, y_train, y_test = train_test_split(symptoms.values.tolist(), results.values.tolist(), test_size=0.2,random_state=20) 

In [80]:
# normalisation train dataset values 
X_train_range = pd.DataFrame(X_train).max() - pd.DataFrame(X_train).min()
X_train_range
X_train_scaled = (pd.DataFrame(X_train) - pd.DataFrame(X_train).min())/(X_train_range)
X_train_scaled

,0,1,2
0,0.252834,0.0,0.452785
1,0.365551,1.0,0.719840
2,0.975454,1.0,0.491346
3,0.133140,1.0,0.847429
4,0.368953,1.0,0.570953
...,...,...,...
19995,0.291160,1.0,0.973013
19996,0.416528,0.0,0.897462
19997,0.040713,0.0,0.797051
19998,0.651129,1.0,0.542628


In [81]:
#Create a svm Classifier
clf = svm.SVC(kernel="rbf",gamma='scale') # rbf Kernel

In [82]:
#Train the model using the training sets
y_train = np.ravel(y_train)

clf.fit(X_train_scaled, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [83]:
# normalisation test dataset values 
X_test_range = (pd.DataFrame(X_test) - pd.DataFrame(X_test).min()).max()
X_test_range
X_test_scaled = (pd.DataFrame(X_test) - pd.DataFrame(X_test).min())/(X_test_range)
X_test_scaled

,0,1,2
0,0.524939,1.0,0.265805
1,0.935205,0.0,0.045115
2,0.194249,1.0,0.997728
3,0.756674,1.0,0.817467
4,0.619889,1.0,0.799443
...,...,...,...
4995,0.342910,0.0,0.877641
4996,0.725470,0.0,0.018602
4997,0.811458,1.0,0.351342
4998,0.227126,1.0,0.887662


In [84]:
#Predict the response for test dataset
y_pred = clf.predict(X_test_scaled)


In [85]:
#accuracy value :
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.975


In [96]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
from seaborn import heatmap

In [99]:
#confusion matrix :
cm =np.array(confusion_matrix(y_test, y_pred, labels =[1,0]))
confusion = pd.DataFrame(cm, index=['tested positive', 'tested negative'], columns=['predicted positive','predicted negative'])
confusion 

,predicted positive,predicted negative
tested positive,2580,60
tested negative,65,2295
